<h1 style = "background-color: rgba(185, 109, 106, 0.3);" > Python Function에 관하여 </h1>

reference : 
1. [Python-클로저(Closure)](http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%81%B4%EB%A1%9C%EC%A0%80-closure/)
2. [1급객체란](https://medium.com/@lazysoul/functional-programming-%EC%97%90%EC%84%9C-1%EA%B8%89-%EA%B0%9D%EC%B2%B4%EB%9E%80-ba1aeb048059)
3. [파이썬 class 및 function namespace 이해하기](https://www.slideshare.net/dahlmoon/pandasdata-frame-20160425-70298865)
<hr>

## 1. First-Class Function

> 파이썬에서는 모든 것을 객체(object)로 다루고 인식한다. 이는 함수도 예외가 아니다. 파이썬에서는 함수를 first-class citizen으로 취급하는데, 이는 아래의 3가지 조건을 만족한다는 얘기다.

1. 변수나 데이타에 할당 할 수 있어야 한다.
2. 객체의 인자로 넘길 수 있어야 한다.
3. 객체의 리턴값으로 리턴 할수 있어야 한다.

In [1]:
def add(x,y):
    return x + y

print(add(2,3))

5


In [2]:
globals()['add'] # 기본적으로 global namespace에서 관리되는 객체이다.

<function __main__.add(x, y)>

#### 1) 변수나 데이터에 할당할 수 있어야 한다.

In [3]:
f = add # add라는 메소드를 f에 담았음
f(2,3)

5

#### 2) 객체의 인자로 넘길 수 있어야 한다.

In [4]:
def square(x):
    return x * x

def cube(x):
    return x * x * x

def quad(x):
    return x * x * x * x

def my_map(func, arg_list):
    result = []
    for i in arg_list:
        result.append(func(i)) # square 함수 호출, func == square
    return result

In [5]:
num_list = [1, 2, 3, 4, 5]

squares = my_map(square, num_list)
cubes = my_map(cube, num_list)
quads = my_map(quad, num_list)

print(squares)
print(cubes)
print(quads)

[1, 4, 9, 16, 25]
[1, 8, 27, 64, 125]
[1, 16, 81, 256, 625]


#### 3) 객체의 리턴값으로 리턴할 수 있어야 한다.

In [6]:
import time

In [7]:
def time_check(func):
    def inner(*args,**kwargs):
        start = time.time()
        result = func(*args,**kwargs)
        print(f"consumed time : {time.time()-start:.7f}")
        return result
    return inner

In [8]:
def add_all(nums):
    result = 0
    for num in nums:
        result += num
    return result

In [9]:
add_all_with_time = time_check(add_all)

In [10]:
add_all([1,2,3,4,5,6,7,8,9,10])

55

In [11]:
add_all_with_time([1,2,3,4,5,6,7,8,9,10])

consumed time : 0.0000021


55

## 2. 클로저(Closure)

> 클로저란 퍼스트클래스 함수를 지원하는 언어의 네임 바인딩 기술이다. 클로저는 어떤 함수를 함수 자신이 가지고 있는 환경과 함께 저장된 레코드다. 또한 함수가 가진 프리변수를 클로저가 만들어지는 당시의 값과 레퍼런스에 맵핑하여 주는 역할을 한다. 클로저는 일반 함수와는 다르게, 자신의 영역 밖에서 호출된 함수의 변수값과 레퍼런스를 복사하고 저장한 뒤, 이 캡처한 값들에 엑세스할 수 있게 도와준다.



In [12]:
def outer_func(name): #1
    message = 'Hi' #3

    def inner_func(): #4
        print("{} say : {}".format(name, message)) #6 

    return inner_func #5

1. (1)에서 정의된 함수 outer_func을 (2)에서 호출합니다.
2. outer_func가 실행된 후, 가장 먼저 하는 것은 message라는 변수에 "hi"라는 문자열을 할당합니다.(3)
3. (4)에서 inner_func을 정의하고 (5)에서 inner_func을 리턴합니다.
4. (6)에서 message 변수를 참조하여 출력합니다. 여기서 message는 inner_func 안에서 정의되지는 않았지만, inner_func 안에서 사용되기 때문에 프리변수라고 부릅니다.

In [13]:
name = '성중쌤'
my_func = outer_func(name)

In [14]:
my_func.__closure__ # Closure cells를 반환. 
                    # Closure Cells이란, 함수를 생성한 환경에서 가져온 값들을 가져옴

(<cell at 0x1095e7738: str object at 0x1095e3260>,
 <cell at 0x1095e72e8: str object at 0x1095b37b0>)

In [15]:
for cell in my_func.__closure__:
    print(cell.cell_contents)

Hi
성중쌤


In [16]:
name = "상재"

def print_name():
    print("my name is {}".format(name))
    
print_name()

my name is 상재


In [17]:
print_name.__closure__

> global Variable은 Closure Cells에 포함되지 않는다. Closure의 특징은 해당 값을 복사하여 내부에 저장하는 데에 있기 때문이다. 그렇기 때문에 위에 정의된 `print_name`은 global variable인 `name`이 바뀌자마자 적용되고, closure cells은 그러지 않다.

In [18]:
name = '선열'
print_name()

my name is 선열


In [19]:
my_func()

성중쌤 say : Hi


이러한 closure의 특징들은 함수를 찍어내서 만들 수 있도록 도와준다. Closure가 없었더라면, 함수를 만드는 함수의 variable들은 서로 공유되어, 독립성을 갖지 못했을 것이다. 하지만 함수에 closure를 지원하게 됨으로써, 우리는 안정적인 코드를 재사용할 수 있게 된다.

<hr>

Copyright(c) 2019 by Public AI. All rights reserved. last updated on 2019/02/04<br>
Writen by PAI, SangJae Kang(rocketgrowthsj@publicai.co.kr) 
<hr>